In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from ingest.catalog import DatasetCatalog
from ingest.loader import RawDatasetLoader
from ingest.fetch.csv import CsvAdapter
from ingest.fetch.sct import SocrataAdapter
from ingest.fetch.excel import ExcelAdapter
from ingest.fetch.metadata import SocrataMetadata

In [14]:
## servicios
catalog = DatasetCatalog()
loader = RawDatasetLoader(
    csv_adapter=CsvAdapter(),
    sct_adapter=SocrataAdapter(),
    excel_adapter=ExcelAdapter(),
)

In [15]:
ds = catalog.get("ciuu_4ac_dane")
records = list(loader.load(ds))
df = pd.DataFrame(records)
df.head(10)

,División,Grupo,Clase,Descripción
0,ESTRUCTURA DETALLADA DE LA CLASIFICACIÓN INDUS...,,,
1,,,,
2,SECCIÓN A,,,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P..."
3,01,,,"Agricultura, ganadería, caza y actividades de ..."
4,,011,,Cultivos agrícolas transitorios
5,,,0111,"Cultivo de cereales (excepto arroz), legumbres..."
6,,,0112,Cultivo de arroz
7,,,0113,"Cultivo de hortalizas, raíces y tubérculos"
8,,,0114,Cultivo de tabaco
9,,,0115,Cultivo de plantas textiles


In [18]:
df_seccion = df[
    df['División']
    .astype(str)
    .str.upper()
    .str.contains("SECCIÓN", na=False)
]
df_seccion

,División,Grupo,Clase,Descripción
2,SECCIÓN A,,,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P..."
48,SECCIÓN B,,,EXPLOTACIÓN DE MINAS Y CANTERAS
74,SECCIÓN C,,,INDUSTRIAS MANUFACTURERAS
278,SECCIÓN D,,,"SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ..."
287,SECCIÓN E,,,DISTRIBUCIÓN DE AGUA; EVACUACIÓN Y TRATAMIENTO...
302,SECCIÓN F,,,CONSTRUCCIÓN
321,SECCIÓN G,,,COMERCIO AL POR MAYOR Y AL POR MENOR; REPARACI...
398,SECCIÓN H,,,TRANSPORTE Y ALMACENAMIENTO
433,SECCIÓN I,,,ALOJAMIENTO Y SERVICIOS DE COMIDA
454,SECCIÓN J,,,INFORMACIÓN Y COMUNICACIONES


In [20]:
## de la fila 48 a la 74
df.iloc[48:75]

,División,Grupo,Clase,Descripción
48,SECCIÓN B,,,EXPLOTACIÓN DE MINAS Y CANTERAS
49,05,,,Extracción de carbón de piedra y lignito
50,,051,0510,Extracción de hulla (carbón de piedra)
51,,052,0520,Extracción de carbón lignito
52,06,,,Extracción de petróleo crudo y gas natural
53,,061,0610,Extracción de petróleo crudo
54,,062,0620,Extracción de gas natural
55,07,,,Extracción de minerales metalíferos
56,,071,0710,Extracción de minerales de hierro
57,,072,,Extracción de minerales metalíferos no ferrosos


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   División     713 non-null    object
 1   Grupo        713 non-null    object
 2   Clase        713 non-null    object
 3   Descripción  713 non-null    object
dtypes: object(4)
memory usage: 22.4+ KB


In [7]:
cols = ["División", "Grupo", "Clase"]

for c in cols:
    df[c] = (
        df[c].astype(str)
             .str.strip()
             .replace(["", "nan", "None"], np.nan)
    )

    df[c] = (
        df[c]
        .str.replace("$", "", regex=False)
        .str.replace(".", "", regex=False)
        .str.replace(",", ".", regex=False)
    )
    df[c] = pd.to_numeric(df[c], errors="coerce")
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   División     88 non-null     float64
 1   Grupo        246 non-null    float64
 2   Clase        495 non-null    float64
 3   Descripción  713 non-null    object 
dtypes: float64(3), object(1)
memory usage: 22.4+ KB


In [8]:
df.describe()


,División,Grupo,Clase
count,88.000000,246.000000,495.000000
mean,490.579545,4614.357724,45391.836364
std,299.508399,2867.821463,28818.807055
min,1.000000,11.000000,111.000000
25%,237.500000,2422.500000,23915.000000
50%,495.000000,4625.000000,46590.000000
75%,742.500000,7087.500000,66135.000000
max,990.000000,9900.000000,99000.000000


In [9]:
df.head()

,División,Grupo,Clase,Descripción
0,NaN,NaN,NaN,
1,NaN,NaN,NaN,
2,NaN,NaN,NaN,"AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y P..."
3,1.0,NaN,NaN,"Agricultura, ganadería, caza y actividades de ..."
4,NaN,11.0,NaN,Cultivos agrícolas transitorios
